# Neuralangelo Data Preperation

Author: Deeepwin   
Date: 31.08.2023   

&#9728;&#65039; Trained with renewable solar power! 

***

### Installation

Install COLMAP from source to utilize graphics card (CUDA). Follow these [instructions](). Use this command:

`cmake .. -GNinja -DCMAKE_CUDA_ARCHITECTURES=native`

### Configuration

Specify data source path:

In [ ]:
# define path to data
DATA_PATH='/mnt/data/github/neuralangelo/datasets/car'

In [ ]:
%cd /mnt/data/github/neuralangelo

### COLMAP Data Processing

Prepare folder structure in your data according to this:

```
+── $DATA_PATH/manually/created/sparse/model
│   +── cameras.txt
│   +── images.txt
│   +── points3D.txt

```

`images.txt` contains the known camera poses, `cameras.txt` the camera type and parameter and `points3D.txt` is empty.

Run sparse reconstruction to generate point clouds. With point cloud bounding box can be definied. Prepare custom data, update camera type and parameters below (take values from `cameras.txt`):

In [ ]:
!colmap feature_extractor \
    --ImageReader.camera_model PINHOLE \
    --ImageReader.camera_params "836.24834503, 839.37481936, 642.0, 481.0" \
    --database_path $DATA_PATH/database.db \
    --image_path $DATA_PATH/images \
    --SiftExtraction.estimate_affine_shape=true \
    --SiftExtraction.domain_size_pooling=true

In [ ]:
!colmap exhaustive_matcher \
    --database_path $DATA_PATH/database.db \
    --SiftMatching.guided_matching=true

This is the most important step before spare reconstruction. Make sure `database.db` and `images.txt` have same image id and file name pairs. To do so, following code helps:

1. Read COLMAP database and and `images.txt` with known poses
2. Sort both according to image file name
3. Copy image ids from database to `images.txt`

In [ ]:
import pandas as pd
from third_party.colmap.scripts.python.database import COLMAPDatabase

db = COLMAPDatabase.connect(DATA_PATH + '/database.db')

# read dtabase table first
df_database = pd.read_sql_query("SELECT * FROM images", db)
db.close()

# read known camera poses
col_names = ['IMAGE_ID', 'QW', 'QX', 'QY', 'QZ', 'TX', 'TY', 'TZ', 'CAMERA_ID', 'NAME']
df_images = pd.read_csv(DATA_PATH + '/manually/created/sparse/model/images.txt', comment='#', sep=' ', names=col_names)

# sort according to images name
df_database = df_database.sort_values(by=['name'])
df_images = df_images.sort_values(by=['NAME'])

# overwrite image ids
df_images['IMAGE_ID'] = df_database['image_id'].values

df_images.to_csv(DATA_PATH + '/manually/created/sparse/model/images.txt', sep=' ', lineterminator='\n\n', index=False, header=False)

Create sparse reconstruction, unselect tri_ignore_two_view_tracks to increase number of 3D points.

In [ ]:
!colmap point_triangulator \
    --Mapper.tri_ignore_two_view_tracks 0 \
    --database_path $DATA_PATH/database.db \
    --image_path $DATA_PATH/images \
    --input_path $DATA_PATH/manually/created/sparse/model \
    --output_path $DATA_PATH/triangulated/sparse/model

To check, if spares model looks good open `colmap gui`:

1. Import sparse model by selecting `Import Model`, select `triangulated/sparse/model` folder
2. Store project name and select `database.db` and `images` folder
3. To view scene, run `Bundle adjustment` with small value for `max_num_iterations` to speed up process

If that looks good, proceed.

It is also possible to create `.bin` model directly from `.txt` model, without sparse reconstruction.

In [ ]:
!python3 third_party/colmap/scripts/python/read_write_model.py \
    --input_model=$DATA_PATH/manually/created/sparse/model --input_format=.txt \
    --output_model=$DATA_PATH/sparse --output_format=.bin

### Neurelangelo Data Processing

Bring data folder in right format, copy `.bin` model from `triangulated/sparse/model` folder to `sparse` folder manually. Rr run model conversion directly from `.txt` model. This way original camera poses will be used:

```
DATA_PATH
├─ database.db      (COLMAP database)
├─ images           (undistorted input images)
├─ images_raw       (raw input images)
├─ sparse           (COLMAP data from SfM)
│  ├─ cameras.bin   (camera parameters)
│  ├─ images.bin    (images and camera poses)
│  ├─ points3D.bin  (sparse point clouds)
│  ├─ 0             (a directory containing individual SfM models. There could also be 1, 2... etc.)
│  ...
├─ stereo (COLMAP data for MVS, not used here)
...
```

Generate transforms.json for tnt or dtu datasets:

In [ ]:
# for tnt just use 'tanks_and_temples' path and have at least two objects in that folder
!bash projects/neuralangelo/scripts/preprocess_tnt.sh $DATA_PATH

Generate transforms.json for custom dataset, loads `.bin` model in `sparse` folder.

In [ ]:
!python3 projects/neuralangelo/scripts/convert_data_to_json.py --data_dir $DATA_PATH --scene_type outdoor

In [ ]:
!python3 projects/neuralangelo/scripts/generate_config.py --sequence_name car --data_dir $DATA_PATH --scene_type outdoor

Let's inspect the results. First, we load the COLMAP data.

In [ ]:
!python3 third_party/colmap/scripts/python/visualize_model.py --input_model=$DATA_PATH/sparse --input_format=.bin

Load `sparse` model and filter points like COLAMP GUI does. Without filtering it might be hard to see the object.

In [ ]:
import numpy as np
import torch
import open3d
import json
import plotly.graph_objs as go
from collections import OrderedDict

# import imaginaire modules.
from projects.nerf.utils import camera, visualize
from third_party.colmap.scripts.python.read_write_model import read_model

# Read the COLMAP data.
cameras, images, points_3D = read_model(path=f"{DATA_PATH}/sparse", ext=".bin")

# convert camera poses.
images = OrderedDict(sorted(images.items()))
qvecs = torch.from_numpy(np.stack([image.qvec for image in images.values()]))
tvecs = torch.from_numpy(np.stack([image.tvec for image in images.values()]))
Rs = camera.quaternion.q_to_R(qvecs)
poses = torch.cat([Rs, tvecs[..., None]], dim=-1)  # [N,3,4]
print(f"# images: {len(poses)}")

# clean up point cloud (like COLMAP GUI does)
pcd = open3d.geometry.PointCloud()
_xyzs = []
_rgbs = []
for point3D in points_3D.values():
    if len(point3D.point2D_idxs) < 3:
        continue
    if point3D.error > 2.0:
        continue
    _xyzs.append(point3D.xyz)
    _rgbs.append(point3D.rgb / 255)

pcd.points = open3d.utility.Vector3dVector(_xyzs)
pcd.colors = open3d.utility.Vector3dVector(_rgbs)

[pcd, _] = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

# Get the sparse 3D points and the colors.
xyzs = torch.from_numpy(np.stack(pcd.points))
rgbs = np.stack(pcd.colors)
# xyzs = torch.from_numpy(np.stack(_xyzs))
# rgbs = np.stack(_rgbs)
# xyzs = torch.from_numpy(np.stack([point.xyz for point in points_3D.values()]))
# rgbs = np.stack([point.rgb for point in points_3D.values()])

rgbs_int32 = (rgbs[:, 0] * 2**16 + rgbs[:, 1] * 2**8 + rgbs[:, 2]).astype(np.uint32)
print(f"# points: {len(xyzs)}")

Visualize filtered point cloud with open3d.

In [ ]:
open3d.visualization.draw_geometries([pcd])

This is where you should visualize and adjust the bounding sphere for Neuralangelo.
- Use the forms to tune `readjust_center` and `readjust_scale` to adjust the bounding sphere.
  - The bounding sphere should ideally *just* encapsulate the target object/scene.
  - In the Lego toy example case, setting `readjust_scale=0.5` would be a good choice.
- Also check whether the camera trajectory matches the expectation from the video observation.

In [ ]:
# Visualize the bounding sphere.
json_fname = f"{DATA_PATH}/transforms.json"
with open(json_fname) as file:
    meta = json.load(file)
center = meta["sphere_center"]
radius = meta["sphere_radius"]
# ------------------------------------------------------------------------------------
# These variables can be adjusted to make the bounding sphere fit the region of interest.
# The adjusted values can then be set in the config as data.readjust.center and data.readjust.scale
readjust_x = 0.  # @param {type:"number"}
readjust_y = -0.9  # @param {type:"number"}
readjust_z = 0.  # @param {type:"number"}
readjust_scale = 0.23  # @param {type:"number"}
readjust_center = np.array([readjust_x, readjust_y, readjust_z])
# ------------------------------------------------------------------------------------
center += readjust_center
radius *= readjust_scale
# Make some points to hallucinate a bounding sphere.
sphere_points = np.random.randn(100000, 3)
sphere_points = sphere_points / np.linalg.norm(sphere_points, axis=-1, keepdims=True)
sphere_points = sphere_points * radius + center

Visualize the bounding sphere in the 3D interactive visualizer.
- If the bounding sphere doesn't look right, readjust in the above form and rerun the code block.
- You can modify `vis_depth` to adjust the size of the cameras.

In [ ]:
vis_depth = 0.2

# visualize with Plotly.
x, y, z = *xyzs.T,
colors = rgbs
sphere_x, sphere_y, sphere_z = *sphere_points.T,
sphere_colors = ["#4488ff"] * len(sphere_points)
traces_poses = visualize.plotly_visualize_pose(poses, vis_depth=vis_depth, xyz_length=0.02, center_size=0.01, xyz_width=0.005, mesh_opacity=0.05)
trace_points = go.Scatter3d(x=x, y=y, z=z, mode="markers", marker=dict(size=0.3, color=colors, opacity=1), hoverinfo="skip")
trace_sphere = go.Scatter3d(x=sphere_x, y=sphere_y, z=sphere_z, mode="markers", marker=dict(size=0.5, color=sphere_colors, opacity=0.3), hoverinfo="skip")
traces_all = traces_poses + [trace_points, trace_sphere]
layout = go.Layout(scene=dict(xaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              yaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              zaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              xaxis_title="X", yaxis_title="Y", zaxis_title="Z", dragmode="orbit",
                              aspectratio=dict(x=1, y=1, z=1), aspectmode="data"), height=800)
fig = go.Figure(data=traces_all, layout=layout)
fig.show()

Colmap rendering and adjusted sphere.


<table><tr>
<td> <img src="pics/car-1.jpg" height="300"/>  </td>
<td> <img src="pics/car-2.jpg" height="300"/>  </td>
</tr></table>

### Plot Mesh

In [ ]:
OUTPUT_DIR=''

In [ ]:
!open3d draw $OUTPUT_DIR